## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-04-18-54-19 +0000,bbc,Putin says Russia will take Donbas by force or...,https://www.bbc.com/news/articles/cgexgwr1wrro...
1,2025-12-04-18-51-52 +0000,nyt,Deadly Attacks in Gaza Test Cease-Fire as Body...,https://www.nytimes.com/2025/12/04/world/middl...
2,2025-12-04-18-46-37 +0000,nyt,New York Times Sues Pentagon Over First Amendm...,https://www.nytimes.com/2025/12/04/business/me...
3,2025-12-04-18-45-26 +0000,bbc,Ireland among countries boycotting Eurovision ...,https://www.bbc.com/news/articles/cde6d8wyp79o...
4,2025-12-04-18-45-22 +0000,nyt,Eurovision Is in Turmoil as Israel’s Participa...,https://www.nytimes.com/2025/12/04/arts/music/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2341/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
42,trump,49
18,new,18
0,putin,16
5,ukraine,14
210,year,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
271,2025-12-04-00-05-45 +0000,nyt,Trump Faces Choices on Russia-Ukraine Talks as...,https://www.nytimes.com/2025/12/03/us/politics...,95
198,2025-12-04-10-00-00 +0000,wsj,Sen. Elizabeth Warren and other Democrats are ...,https://www.wsj.com/politics/policy/democrats-...,94
15,2025-12-04-18-18-38 +0000,wapo,Putin maintains hard line in talks as Trump sa...,https://www.washingtonpost.com/world/2025/12/0...,89
208,2025-12-04-09-00-00 +0000,cbc,Trump offered Putin a tactical victory in Ukra...,https://www.cbc.ca/news/world/trump-putin-envo...,87
75,2025-12-04-16-15-58 +0000,wapo,Trump’s crackdown on Russian oil hangs heavy a...,https://www.washingtonpost.com/world/2025/12/0...,85


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
271,95,2025-12-04-00-05-45 +0000,nyt,Trump Faces Choices on Russia-Ukraine Talks as...,https://www.nytimes.com/2025/12/03/us/politics...
275,83,2025-12-04-00-00-00 +0000,wsj,Defense Secretary Pete Hegseth shocked officia...,https://www.wsj.com/politics/national-security...
252,60,2025-12-04-02-26-10 +0000,48hills,Under the Stars: Fresh new music nods to wrap ...,https://48hills.org/2025/12/under-the-stars-fr...
161,55,2025-12-04-11-53-24 +0000,nypost,Admiral who ordered September boat strike will...,https://nypost.com/2025/12/04/us-news/admiral-...
103,45,2025-12-04-15-04-19 +0000,nypost,Putin’s alleged secret daughter tells Ukrainia...,https://nypost.com/2025/12/04/world-news/putin...
198,45,2025-12-04-10-00-00 +0000,wsj,Sen. Elizabeth Warren and other Democrats are ...,https://www.wsj.com/politics/policy/democrats-...
257,44,2025-12-04-01-55-16 +0000,nypost,Halle Berry bashes Gov. Gavin Newsom at NYT su...,https://nypost.com/2025/12/03/us-news/halle-be...
168,43,2025-12-04-11-02-24 +0000,bbc,Israel says Palestinian groups returned body o...,https://www.bbc.com/news/articles/cd6x2j4g3z7o...
45,41,2025-12-04-17-33-00 +0000,wsj,The FBI arrested a Virginia man in connection ...,https://www.wsj.com/politics/national-security...
368,39,2025-12-03-20-27-24 +0000,nypost,"Pentagon watchdog finds Hegseth put US troops,...",https://nypost.com/2025/12/03/us-news/pentagon...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
